Optical fibers are used to transmit photons around, e.g., between repeaters.

Physically, a photon passing through an optical fiber can be modelled by a quantum channel acting on the state of the qubit.

The optical fiber is used in this scenario: repeater 1 wants to share entanglement with repeater 2. So repeater 1 prepares an EPR pair locally, then sends one half of the pair to repeater 2 through the quantum channel that models the passage of a photon through the fiber. The channel is applied to the qubit (in the global state container) in the function `carryQubit` below. For now we model the photon transmission by an identity map for simplicity until the simualtor is built up a bit.

In [46]:
%%writefile optical_fiber.py
import sys
import math
from qutip import *

sys.path.append("..")
from common.global_state_container import global_state_container
from ..qubit_carriers.photon import Photon

class OpticalFiber(object):
    def __init__(self, length=1):
        print("creating new optical fiber")
        self.global_state = global_state_container.state
#         self.id = None
        # The length of the fiber.
        self.length = length
        # The two nodes at the two ends of the fiber.
        # A node can be a repeaterHardware, an end node, a heralding station, 
        # etc. For now it's a repeaterHardware object for simplicity.
        self.node1 = None
        self.node2 = None
#         self.photon12 = Photon(self)   # this is the photon going from node 1 to node 2
#         self.photon21 = Photon(self)

    def connect_node_hardware(self, node_hardware, pos=None):
        if pos == 1:
            self.node1 = node_hardware
        elif pos == 2:
            self.node2 = node_hardware
        else:
            if self.node1 is None:
                self.node1 = node_hardware
            else:
                self.node2 = node_hardware
                
    def is_connected(self, node_hardware):
        return node_hardware == self.node1 or node_hardware == self.node2

    def carry_photon(self, photon, sender):  # Here there will be a quantum channel applied to the state
        # sender is a node hardware object.
        # apply channel here
        # for now make it the identity channel
        gate = tensor([identity(2) for _ in range(int(math.log2(self.global_state.state.shape[0])))])
        new_state = gate * self.global_state.state * gate.dag()  # for now the map is a identity unitary gate
        self.global_state.update_state(new_state)
        # send output state to the receiver
        receiver = self.node2 if sender == self.node1 else self.node1
        receiver.receive_photon_from_fiber(photon, self)

Overwriting optical_fiber.py


A channel that can be used to model an optical fiber is the Gaussian bosonic channel [CITATION NEEDED]. The capacity of this channel is ...